In [61]:
import pandas as pd
import numpy as np
import os
import rebound

path = '/scratch/dtamayo/'
icpath = path +'random/initial_conditions/runs/ic'
fcpath = path +'random/final_conditions/runs/fc'

df = pd.read_csv(path+'random/random.csv', index_col=0)

simIDmax = 15000
df = df.head(simIDmax)
df.tail()

,runstring
14995,0014995.bin
14996,0014996.bin
14997,0014997.bin
14998,0014998.bin
14999,0014999.bin


In [62]:
columns = ['Stable','instability_time','Rel_Eerr','RHill12','RHill23','beta12', 'beta23']
for i in ['1', '2', '3']:
    columns += ['m'+i, 'a'+i, 'P'+i, 'e'+i, 'pomega'+i, 'inc'+i, 'Omega'+i, 'f'+i]

In [63]:
def get_initial_orbital_elements(row):
    try:
        sim = rebound.Simulation.from_file(icpath+row["runstring"])
        simf = rebound.Simulation.from_file(fcpath+row["runstring"])
        ps = sim.particles
        stable = np.isclose(simf.t, 1.e9)
        instability_time = simf.t
        Rel_Eerr = abs((simf.calculate_energy()-sim.calculate_energy())/sim.calculate_energy())
        RHill12 = ((ps[1].m + ps[2].m)/(3.*ps[0].m))**(1./3.)
        RHill23 = ((ps[2].m + ps[3].m)/(3.*ps[0].m))**(1./3.)
        beta12 = (ps[2].a - ps[1].a)/RHill12
        beta23 = (ps[3].a - ps[2].a)/RHill23
        features = [stable,instability_time,Rel_Eerr,RHill12,RHill23,beta12,beta23]
        for i in [1,2,3]:
            features += [ps[i].m, ps[i].a, ps[i].P, ps[i].e, ps[i].pomega, ps[i].inc, ps[i].Omega, ps[i].f]
        return pd.Series(features, index=columns)    
    except:
        return np.nan

In [64]:
%%time
df = pd.concat([df, df.apply(get_initial_orbital_elements, axis=1)], axis=1)

/mnt/raid-cita/dtamayo/stability/lib/python3.4/site-packages/rebound/simulation.py:374: RuntimeWarning: You have to reset function pointers after creating a reb_simulation struct with a binary file.
  warnings.warn(message, RuntimeWarning)


CPU times: user 16 s, sys: 496 ms, total: 16.5 s
Wall time: 17 s


In [65]:
df.tail(10)

,runstring,Stable,instability_time,Rel_Eerr,RHill12,RHill23,beta12,beta23,m1,a1,...,Omega2,f2,m3,a3,P3,e3,pomega3,inc3,Omega3,f3
14990,0014990.bin,False,8.881944e+06,6.970683e-09,0.008035,0.024971,5.932913,18.981663,1.447016e-06,1.0,...,1.463629,-2.044393,4.660160e-05,1.521657,1.877002,0.000071,-3.597323,0.093795,-1.163587,2.145837
14991,0014991.bin,False,4.105794e+08,1.520917e-06,0.009773,0.014433,20.406519,7.767186,2.592861e-06,1.0,...,2.300958,-3.761123,8.813249e-06,1.311537,1.501995,0.000907,-3.791939,0.009719,-0.892216,4.491973
14992,0014992.bin,False,1.163938e+02,7.757771e-05,0.007920,0.020145,29.846190,3.193360,1.747540e-07,1.0,...,-2.648553,-3.762293,2.320896e-05,1.300719,1.483440,0.004665,3.793154,0.053988,1.739732,-4.699110
14993,0014993.bin,False,1.965516e+03,1.213398e-08,0.030624,0.017393,14.174662,10.058129,8.602002e-05,1.0,...,-2.441854,2.306595,1.564621e-05,1.609016,2.040885,0.003507,3.099068,0.079411,1.602544,-2.808318
14994,0014994.bin,False,3.852079e+01,1.440579e-06,0.020929,0.022484,27.235893,1.959445,1.226492e-06,1.0,...,2.861755,3.354885,7.822895e-06,1.614075,2.050586,0.030347,-3.564980,0.026291,-2.614749,3.945627
14995,0014995.bin,True,1.000000e+09,4.247491e-09,0.008212,0.017684,8.745778,30.425524,1.295143e-06,1.0,...,-2.064089,1.625026,1.622396e-05,1.609862,2.042581,0.018861,-1.948353,0.017213,-2.700617,1.631530
14996,0014996.bin,False,1.143031e+06,3.488275e-07,0.013966,0.019213,7.373403,24.867761,8.521014e-07,1.0,...,0.738024,-1.125991,1.395751e-05,1.580777,1.987472,0.132649,-0.279699,0.007771,-2.223644,-4.886753
14997,0014997.bin,True,1.000000e+09,1.140037e-08,0.027045,0.019439,29.303032,22.854837,3.779511e-05,1.0,...,0.612557,-3.595167,4.853386e-07,2.236787,3.345215,0.039203,2.326575,0.008720,0.118470,-2.144004
14998,0014998.bin,True,1.000000e+09,1.345430e-09,0.032227,0.033911,17.694817,37.064336,3.577833e-06,1.0,...,-0.751806,-1.152337,2.015521e-05,2.827155,4.753334,0.003423,-1.590951,0.002031,-0.120461,0.849382
14999,0014999.bin,False,1.511689e+02,1.251915e-05,0.027526,0.013841,6.786493,25.085193,5.908206e-05,1.0,...,1.520140,-0.536341,4.465852e-06,1.534015,1.899895,0.000654,1.191751,0.011744,2.858557,1.581108


In [68]:
df.to_csv('../csvs/initial_orbital_elements.csv', encoding='ascii')